This would be a Python script you set up on a Cron Job (or something similar), but for now I'm testing in a notebook.

In [ ]:
# I wanted to use requests and beautifulsoup for this, but it still wont work because the site
# loads what I'm getting from javascript it seems. So back to Selenium (well undetected_chromedriver)
# import requests
# from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from undetected_chromedriver import By
import datetime
import sqlite3
from datetime import timezone
import json
import os
import time
# from zoneinfo import ZoneInfo

# Parameters
cookies_path = "cookies.json"

options = uc.ChromeOptions()
options.add_argument("--headless")
driver = uc.Chrome(use_subprocess=False, options=options)

You will need to launch this without headless the first time to log in and get the cookies for Udemy (or find another way to export them and save them as a json file to load in)

In [ ]:
def saveCookies(t_driver: uc.Chrome, f_path: str, force_cookies: bool = True) -> None:
    '''
    Saves the cookies from your udemy in

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param f_path: The file name (or full file path)
    :type f_path: str

    :param force_cookies: If we force the cookies even if they already exist, defaults to [True]
    :type force_cookies: bool

    :return: Nothing
    :rtype: None
    '''
    # Checking if the cookies file already exists
    if(force_cookies or not os.path.exists(f_path)):
        # Wait for an input for the user to log in
        input("Continue after you've logged into Udemy!")
        # To save cookies
        saved_cookies = t_driver.get_cookies()
        with open(f_path, "w") as file:
            json.dump(saved_cookies, file)
    return None
def loadCookies(t_driver: uc.Chrome, f_path: str):
    '''
    Loads your Udemy cookies in

    :param t_driver: The web driver
    :type t_driver: Chrome

    :param f_path: The file name (or full file path)
    :type f_path: str

    :return: Nothing
    :rtype: None
    '''
    # To load in cookies
    with open(f_path, "r") as file:
        saved_cookies = json.load(file)
    for x in saved_cookies:
        t_driver.add_cookie(x)
    # Wait for one second just to be sure (probably not needed)
    time.sleep(1)
    return None

In [ ]:
con = sqlite3.connect("udemy_sales.db")
cur = con.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS UdemySales(userType TEXT, salePrice REAL, date TEXT, hour INTEGER, dateTimeUTC TEXT);")
con.commit()

In [ ]:
for x in range(2):
    # Initialize variables in case of failure
    sale_price = None
    cur_dt = datetime.datetime.now(tz=timezone.utc)
    cur_date = cur_dt.strftime("%Y-%m-%d")
    cur_hour = cur_dt.hour
    cur_dt = cur_dt.strftime("%Y-%m-%d %H:%M:%S.%f")[:-2]
    if x == 0:
        # Delete any possible cookies
        driver.delete_all_cookies()
        mode = "New_User"
    else:
        # Load user from cookies
        loadCookies(driver, "cookies.json")
        mode = "Existing_User"
    try:
        # Go to udemy
        driver.get("https://www.udemy.com/")
        time.sleep(5) # Waiting just to be sure the Javascript loads
        sale_bar = driver.find_element(By.XPATH, "//section[@aria-label='Marketing notification from Udemy']")
        sale_bar = sale_bar.find_element(By.XPATH, ".//span[@data-purpose='smart-bar-subtitle']").text
        # I wanted to use Decimal because it's money, but SQLite doesn't support it so floats it is
        #sale_price = Decimal(sale_bar.split("$")[1].split(" ")[0][:-1])
        sale_price = float(sale_bar.split("$")[1].split(" ")[0][:-1])
    except:
        pass
    cur.execute("INSERT INTO UdemySales VALUES(?, ?, ?, ?, ?);", (mode, sale_price, cur_date, cur_hour, cur_dt))
# Close connections
con.commit()
cur.close()
driver.close()
